# **Mini GPT Implementation**

This notebook implements and experiments the Generative Pretrained Transformer (GPT) series (e.g., GPT-2, GPT-3). The model architecture is the foundational building block underlying the GPT family, and its design is inspired by a major breakthrough in deep learning, natural language processing and machine translation: the **self-attention mechanism** introduced in **“Attention Is All You Need”** by Vaswani et al.

The models were trained on Moby-Dick book only for experimental purposes.

In [62]:
!pip install --quiet --upgrade pypdf tiktoken

In [47]:
import os
from pypdf import PdfReader
import re

### **Load the book data that will be used to train the model**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
pdf_path = "/content/drive/MyDrive/Datasets/Book/Outliers_Malcolm_Gladwell.pdf"

In [17]:
def pdf_to_text(pdf_path, output_txt_path=None, save_to_file=True):
    """reads pdf and converts to text, optionally saving to .txt file."""

    # check if pdf exists.
    if not os.path.exists(pdf_path):
        print(f"error: '{pdf_path}' not found.")
        return None

    try:
        # open pdf in binary mode.
        with open(pdf_path, 'rb') as file:

            # create pdf reader.
            reader = PdfReader(file)
            num_pages = len(reader.pages)
            print(f"processing {num_pages} pages...")

            # extract all text.
            full_text = []
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text = page.extract_text()

                if text:
                    full_text.append(text)
                else:
                    print(f"warning: page {page_num + 1} had no extractable text.")

            # combine all pages.
            combined_text = "\n\n".join(full_text)

            # save to file if requested.
            if save_to_file:
                if output_txt_path is None:
                    output_txt_path = pdf_path.replace('.pdf', '.txt')

                with open(output_txt_path, 'w', encoding='utf-8') as txt_file:
                    txt_file.write(combined_text)

                print(f"saved to: {output_txt_path}")

            return combined_text

    except Exception as e:
        print(f"error reading pdf: {e}")
        return None

In [18]:
def batch_pdf_to_text(folder_path, output_folder=None):
    """converts all pdfs in a folder to text files."""

    # check if folder exists.
    if not os.path.exists(folder_path):
        print(f"error: folder '{folder_path}' not found.")
        return

    # create output folder if needed.
    if output_folder and not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # get all pdf files.
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

    if not pdf_files:
        print("no pdf files found in folder.")
        return

    print(f"found {len(pdf_files)} pdf files. converting...")

    # process each pdf.
    for pdf_file in pdf_files:
        pdf_path = os.path.join(folder_path, pdf_file)

        if output_folder:
            output_path = os.path.join(output_folder, pdf_file.replace('.pdf', '.txt'))
        else:
            output_path = None

        print(f"\nprocessing: {pdf_file}")
        pdf_to_text(pdf_path, output_path)

    print("\nbatch conversion complete!")

In [22]:
raw_text_content = pdf_to_text(pdf_path)

processing 249 pages...
saved to: /content/drive/MyDrive/Datasets/Book/Outliers_Malcolm_Gladwell.txt


In [23]:
print("Total number of character:", len(raw_text_content))
print(raw_text_content[:99])

Total number of character: 460995
OUTLIERS
The Story of Success
MALCOLM GLADWELL
BACK BAY BOOKS
LITTLE, BROWN AND COMPANY
NEW YORK   


In [51]:
def preprocess_text(raw_text, max_preview=30):
    """splits text on punctuation and whitespace, removes empty strings."""

    # split on common punctuation and whitespace.
    pattern = r'([,.:;?_!"()\'\[\]{}\/\\|—–-]+|\.\.\.|\s+)'

    tokens = re.split(pattern, raw_text)

    # remove whitespace and filter empty strings.
    preprocessed = [item.strip() for item in tokens if item.strip()]

    # preview first n tokens.
    if max_preview:
        print(f"first {max_preview} tokens:")
        print(preprocessed[:max_preview])
        print(f"\ntotal word level tokens: {len(preprocessed)}\n")

    return preprocessed

In [92]:
preprocessed = preprocess_text(raw_text_content, max_preview=30)

# get the vocabulary size of the dataset.
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

first 30 tokens:
['OUTLIERS', 'The', 'Story', 'of', 'Success', 'MALCOLM', 'GLADWELL', 'BACK', 'BAY', 'BOOKS', 'LITTLE', ',', 'BROWN', 'AND', 'COMPANY', 'NEW', 'YORK', '•', 'BOSTON', '•', 'LONDON', 'Begin', 'Reading', 'Table', 'of', 'Contents', 'Reading', 'Group', 'Guide', 'Copyright']

total word level tokens: 92765

11327


In [96]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [97]:
class WordTokenizer:
    """tokenizes text into ids and decodes ids back to text. it focuses on word-level tokenization."""

    def __init__(self, vocab):
        """initializes tokenizer with vocabulary mappings."""

        self.tok_to_int = vocab
        self.int_to_tok = {integer: token for token, integer in vocab.items()}
        self.pattern = r'([,.:;?_!"()\'\[\]{}\/\\|—–-]+|\.\.\.|\s+)'

    def encode(self, text):
        """converts text to list of token ids."""

        # split on punctuation and whitespace.
        preprocessed = re.split(self.pattern, text)

        # remove empty strings and whitespace.
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        # replace unknown tokens with <|unk|>.
        preprocessed = [token if token in self.tok_to_int else "<|unk|>" for token in preprocessed]

        # convert tokens to ids.
        ids = [self.tok_to_int[tok] for tok in preprocessed]

        # return ids
        return ids


    def decode(self, ids):
        """converts list of token ids back to text."""

        # map ids to tokens.
        tokens = [self.int_to_tok[id] for id in ids]

        # join tokens with spaces.
        text = " ".join(tokens)

        # remove spaces before punctuation.
        text = re.sub(self.pattern, r'\1', text)

        # remove spaces before punctuation.
        text = re.sub(r'\s+([,.:;?_!"()\'\[\]{}\/\\|—–-])', r'\1', text)

        return text

In [99]:
# initialize the wordtokenizer class.
tokenizer = WordTokenizer(vocab)

# sample text from (and outside) outliers book.
text1 = "Chris Langan's mother was from San Francisco and was estranged from her family."

# sample text with unknown words for testing.
text2 = "do you know about smartphone cryptocurrency?"


# combine texts with special separator token.
text = " <|endoftext|> ".join((text1, text2))

print(f"original text: {text}")

# convert text to token ids.
encoded_text = tokenizer.encode(text)
print(f"\nencoded text: {encoded_text}")

# convert token ids back to text.
decoded_text = tokenizer.decode(encoded_text)
print(f"\ndecoded text: {decoded_text}")

original text: Chris Langan's mother was from San Francisco and was estranged from her family. <|endoftext|> do you know about smartphone cryptocurrency?

encoded text: [915, 1905, 11328, 8970, 7500, 10584, 5993, 2676, 1342, 3561, 10584, 5524, 5993, 6370, 5700, 29, 11328, 11328, 11328, 11328, 11328, 5192, 10838, 6940, 3291, 11328, 11328, 430]

decoded text: Chris Langan <|unk|> s mother was from San Francisco and was estranged from her family. <|unk|> <|unk|> <|unk|> <|unk|> <|unk|> do you know about <|unk|> <|unk|>?


### **Byte Pair Encoding (BPE) Tokenizer.**

The GPT Series uses a more sophisticated encoding preprocessing technique to convert human language (texts or words) into sub-word units for its training processes. It uses the Byte Pair Encoding (BPE) tokenizer. BPE tokenizer is fast, performant and compressed compared to other tokenization methods.

Its implementation is quite complex and time-consuming, so I used the tiktoken open-source library to implement this. tiktoken is an invertible and lossless tokenization technique. it can work on arbitrary text - even text that is not present in the tokenizer's training data. it compresses the text, whereby making the token sequence shorter than the bytes corresponding to the original text.

It also attempts to let the model see common subwords. For instance, "ing" is a common subword in English, so BPE encodings will often split "encoding" into tokens like "encod" and "ing" (instead of e.g. "enc" and "oding"). Because the model will then see the "ing" token again and again in different contexts, it helps in better contextual understanding of grammar and in generalization of models.